# Day11_2: 감성 분석 (Sentiment Analysis)

## 📚 학습 목표

**Part 1: 사전 기반 감성 분석**
1. 감성 분석의 개념과 비즈니스 활용 이해하기
2. VADER 감성 분석기로 영어 텍스트 분석하기
3. KNU 한국어 감성 사전 활용법 배우기
4. 커스텀 감성 사전 구축하기
5. 긍정/부정/중립 분류 구현하기

**Part 2: 감성 분석 시각화와 응용**
1. pandas로 감성 점수 집계 및 분석하기
2. Plotly로 감성 분포 시각화하기 (파이/막대 차트)
3. 시계열 감성 트렌드 분석하기
4. 머신러닝 기반 감성 분석 소개

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 감성 분석 | 고객 반응 파악 | "이 제품에 대한 고객 의견이 긍정적인가?" |
| 사전 기반 방법 | 빠른 초기 분석 | 라벨링 없이 즉시 감성 점수 산출 |
| 감성 시각화 | 인사이트 전달 | 브랜드별 감성 비교 대시보드 |
| 트렌드 분석 | 위기 감지 | SNS 부정 여론 급증 모니터링 |

**분석가 관점**: 감성 분석은 수천 개의 리뷰를 한눈에 파악하게 해주는 마법 같은 도구입니다!

---

# Part 1: 사전 기반 감성 분석

---

## 1.1 감성 분석이란?

### 정의

**감성 분석(Sentiment Analysis)**은 텍스트에서 **의견, 감정, 태도**를 자동으로 추출하는 자연어 처리 기술입니다.

### 감성 분석의 유형

| 유형 | 설명 | 예시 |
|------|------|------|
| 극성 분류 | 긍정/부정/중립 분류 | "좋아요" → 긍정 |
| 감정 분류 | 기쁨, 슬픔, 분노 등 | "화가 난다" → 분노 |
| 강도 측정 | 감성의 강도 점수화 | "매우 좋아" > "좋아" |
| 측면 기반 | 특정 속성별 감성 | "음식은 좋지만 서비스가 별로" |

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import re
from collections import Counter
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# VADER 감성 분석기 (영어용)
# pip install vaderSentiment
try:
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    vader_available = True
    print("VADER 감성 분석기 로드 완료")
except ImportError:
    vader_available = False
    print("VADER 미설치. pip install vaderSentiment 로 설치하세요.")

print("라이브러리 임포트 완료")

### 감성 분석 방법론 비교

| 방법 | 장점 | 단점 | 적합한 상황 |
|------|------|------|------------|
| 사전 기반 | 라벨링 불필요, 빠름 | 도메인 한정적 | 초기 분석, 일반 텍스트 |
| 규칙 기반 | 해석 가능, 커스텀 | 규칙 작성 노력 | 특정 도메인 |
| ML 기반 | 높은 정확도 | 대량 라벨 데이터 필요 | 정밀 분석 |
| 딥러닝 | 문맥 이해 | 고비용, 복잡함 | 대규모 서비스 |

---

## 1.2 VADER 감성 분석 (영어)

### VADER란?

**VADER (Valence Aware Dictionary and sEntiment Reasoner)**는 소셜 미디어 텍스트에 특화된 규칙 기반 감성 분석 도구입니다.

### 특징

- 이모티콘, 대문자, 느낌표 등 SNS 특성 반영
- 강조어(very, extremely) 처리
- 부정어(not, never) 반전 처리
- -1 (매우 부정) ~ +1 (매우 긍정) 점수

In [ ]:
# VADER 감성 분석 예시
if vader_available:
    analyzer = SentimentIntensityAnalyzer()
    
    # 테스트 문장들
    test_sentences = [
        "I love this product! It's amazing!",
        "This is the worst experience ever.",
        "The movie was okay, nothing special.",
        "AMAZING!!! Best purchase I've ever made!!!",
        "Not bad, but could be better."
    ]
    
    print("VADER 감성 분석 결과")
    print("=" * 70)
    
    for sentence in test_sentences:
        scores = analyzer.polarity_scores(sentence)
        print(f"\n문장: {sentence}")
        print(f"  긍정: {scores['pos']:.3f}, 부정: {scores['neg']:.3f}, 중립: {scores['neu']:.3f}")
        print(f"  종합 점수 (compound): {scores['compound']:.3f}")
else:
    print("VADER가 설치되지 않았습니다.")

### 💡 실무 예시: 제품 리뷰 감성 분류

In [ ]:
# 제품 리뷰 데이터 (영어)
product_reviews = pd.DataFrame({
    'review_id': range(1, 11),
    'product': ['Laptop', 'Laptop', 'Phone', 'Phone', 'Phone', 
                'Headphones', 'Headphones', 'Tablet', 'Tablet', 'Tablet'],
    'review': [
        "Excellent laptop! Fast and reliable.",
        "Battery life is terrible. Very disappointed.",
        "Great phone, love the camera!",
        "Screen cracked after 2 weeks. Awful quality.",
        "Average phone. Does the job.",
        "Best headphones I've ever owned! Amazing sound!",
        "Comfortable but sound quality is mediocre.",
        "Perfect for reading and browsing.",
        "Too slow. Not worth the money.",
        "Good tablet, fast delivery."
    ]
})

print("제품 리뷰 데이터")
print(product_reviews)

In [ ]:
# VADER로 감성 분석 수행
def analyze_sentiment_vader(text):
    """VADER를 사용한 감성 분석"""
    if not vader_available:
        return {'compound': 0, 'pos': 0, 'neg': 0, 'neu': 1}
    scores = analyzer.polarity_scores(text)
    return scores

def classify_sentiment(compound_score):
    """compound 점수로 감성 분류"""
    if compound_score >= 0.05:
        return '긍정'
    elif compound_score <= -0.05:
        return '부정'
    else:
        return '중립'

# 감성 분석 적용
if vader_available:
    product_reviews['sentiment_scores'] = product_reviews['review'].apply(analyze_sentiment_vader)
    product_reviews['compound'] = product_reviews['sentiment_scores'].apply(lambda x: x['compound'])
    product_reviews['sentiment'] = product_reviews['compound'].apply(classify_sentiment)
    
    print("감성 분석 결과")
    print(product_reviews[['product', 'review', 'compound', 'sentiment']])

---

## 1.3 한국어 감성 분석 - KNU 감성 사전

### KNU 한국어 감성 사전

군산대학교(KNU)에서 개발한 한국어 감성 사전으로, 긍정/부정 단어와 점수가 포함되어 있습니다.

### 감성 사전 구조

```
단어, 점수
좋다, 2
싫다, -2
그저그렇다, 0
```

In [ ]:
# 간단한 한국어 감성 사전 (KNU 스타일)
# 실제로는 https://github.com/park1200656/KnuSentiLex 에서 다운로드

korean_sentiment_dict = {
    # 긍정 단어 (점수: 1 ~ 2)
    '좋다': 2, '좋아': 2, '좋은': 2, '좋아요': 2,
    '훌륭하다': 2, '훌륭한': 2, '훌륭해요': 2,
    '최고': 2, '최고다': 2, '최고예요': 2,
    '만족': 2, '만족스럽다': 2, '만족해요': 2,
    '추천': 1, '추천해요': 1, '추천합니다': 1,
    '맛있다': 2, '맛있어요': 2, '맛있는': 2,
    '친절하다': 2, '친절해요': 2, '친절한': 2,
    '빠르다': 1, '빨라요': 1, '빠른': 1,
    '편하다': 1, '편해요': 1, '편한': 1,
    '깨끗하다': 1, '깨끗해요': 1, '깨끗한': 1,
    '행복': 2, '기쁘다': 2, '즐겁다': 2,
    '감사': 1, '고맙다': 1, '감동': 2,
    
    # 부정 단어 (점수: -1 ~ -2)
    '나쁘다': -2, '나빠요': -2, '나쁜': -2,
    '싫다': -2, '싫어요': -2, '싫은': -2,
    '별로': -1, '별로다': -1, '별로예요': -1,
    '실망': -2, '실망스럽다': -2, '실망이에요': -2,
    '불만족': -2, '불만': -2,
    '최악': -2, '최악이다': -2,
    '느리다': -1, '느려요': -1, '느린': -1,
    '불친절': -2, '불친절하다': -2,
    '불편하다': -1, '불편해요': -1,
    '더럽다': -1, '더러워요': -1,
    '짜증': -2, '화나다': -2, '슬프다': -1,
    '후회': -2, '아쉽다': -1,
    
    # 강조어
    '매우': 1.5, '정말': 1.5, '너무': 1.3, '아주': 1.3,
    '진짜': 1.3, '완전': 1.5, '엄청': 1.5,
    
    # 부정어
    '안': -1, '않다': -1, '못': -1, '없다': -0.5
}

print(f"한국어 감성 사전 크기: {len(korean_sentiment_dict)}개 단어")
print("\n긍정 단어 예시:", [k for k, v in korean_sentiment_dict.items() if v > 0][:5])
print("부정 단어 예시:", [k for k, v in korean_sentiment_dict.items() if v < 0][:5])

In [ ]:
def analyze_korean_sentiment(text, sentiment_dict):
    """
    한국어 텍스트 감성 분석 (사전 기반)
    
    Parameters:
    - text: 분석할 텍스트
    - sentiment_dict: 감성 사전 (단어: 점수)
    
    Returns:
    - dict: 감성 점수, 분류, 발견된 키워드
    """
    # 텍스트 전처리
    text = text.lower()
    
    total_score = 0
    found_words = []
    multiplier = 1.0
    
    # 강조어 확인
    intensifiers = ['매우', '정말', '너무', '아주', '진짜', '완전', '엄청']
    for word in intensifiers:
        if word in text:
            multiplier = sentiment_dict.get(word, 1.0)
            break
    
    # 부정어 확인
    negation = False
    if '안 ' in text or '않' in text or '못 ' in text:
        negation = True
    
    # 감성 단어 검색
    for word, score in sentiment_dict.items():
        if word in text and word not in intensifiers and word not in ['안', '않다', '못', '없다']:
            # 강조어 곱하기
            adjusted_score = score * multiplier
            # 부정어 있으면 반전
            if negation:
                adjusted_score = -adjusted_score
            total_score += adjusted_score
            found_words.append((word, score))
    
    # 정규화 (-1 ~ 1)
    if found_words:
        normalized_score = total_score / (len(found_words) * 2)  # 최대 점수 2 기준
        normalized_score = max(-1, min(1, normalized_score))  # -1 ~ 1 클리핑
    else:
        normalized_score = 0
    
    # 분류
    if normalized_score >= 0.1:
        sentiment = '긍정'
    elif normalized_score <= -0.1:
        sentiment = '부정'
    else:
        sentiment = '중립'
    
    return {
        'score': normalized_score,
        'raw_score': total_score,
        'sentiment': sentiment,
        'keywords': found_words
    }

# 테스트
test_korean = [
    "이 제품 정말 좋아요! 추천합니다.",
    "배송이 너무 느리고 불친절해요.",
    "그냥 그래요. 보통이에요.",
    "음식이 매우 맛있어요! 최고!",
    "서비스가 안 좋아요."
]

print("한국어 감성 분석 결과")
print("=" * 60)

for text in test_korean:
    result = analyze_korean_sentiment(text, korean_sentiment_dict)
    print(f"\n문장: {text}")
    print(f"  점수: {result['score']:.3f}, 분류: {result['sentiment']}")
    print(f"  키워드: {result['keywords']}")

### 💡 실무 예시: 쇼핑몰 리뷰 분석

In [ ]:
# 쇼핑몰 리뷰 데이터 (한국어)
korean_reviews = pd.DataFrame({
    'review_id': range(1, 16),
    'product': ['스마트폰'] * 5 + ['노트북'] * 5 + ['이어폰'] * 5,
    'review': [
        # 스마트폰 리뷰
        "카메라 화질이 정말 좋아요! 만족합니다.",
        "배터리가 너무 빨리 닳아요. 실망이에요.",
        "디자인은 예쁜데 가격이 좀 비싸요.",
        "완전 최고! 추천합니다!",
        "그냥 그래요. 기대했던 것보다 별로.",
        # 노트북 리뷰
        "가볍고 빨라서 좋아요. 매우 만족!",
        "발열이 심하고 팬 소음이 커요.",
        "가성비 최고! 추천해요.",
        "키보드가 불편해요. 후회됩니다.",
        "업무용으로 딱 좋아요.",
        # 이어폰 리뷰
        "음질이 정말 훌륭해요!",
        "착용감이 불편하고 연결이 자주 끊겨요.",
        "가격 대비 괜찮아요.",
        "최악이에요. 돈 아깝다.",
        "노이즈캔슬링이 좋아요. 추천!"
    ],
    'rating': [5, 2, 3, 5, 3, 5, 2, 5, 1, 4, 5, 2, 4, 1, 5]
})

print("쇼핑몰 리뷰 데이터")
print(korean_reviews)

In [ ]:
# 한국어 감성 분석 적용
korean_reviews['sentiment_result'] = korean_reviews['review'].apply(
    lambda x: analyze_korean_sentiment(x, korean_sentiment_dict)
)
korean_reviews['sentiment_score'] = korean_reviews['sentiment_result'].apply(lambda x: x['score'])
korean_reviews['sentiment'] = korean_reviews['sentiment_result'].apply(lambda x: x['sentiment'])
korean_reviews['keywords'] = korean_reviews['sentiment_result'].apply(lambda x: [w[0] for w in x['keywords']])

print("한국어 감성 분석 결과")
print(korean_reviews[['product', 'review', 'rating', 'sentiment_score', 'sentiment']])

---

## 1.4 커스텀 감성 사전 구축

### 도메인 특화 사전의 중요성

일반 감성 사전은 도메인 특화 용어를 처리하지 못합니다.

예시:
- 금융: "폭등" (일반적으로 부정, 주식에서는 긍정)
- 게임: "킬" (일반적으로 부정, 게임에서는 중립)
- 음식: "매콤하다" (호불호 있음)

In [ ]:
# 도메인 특화 감성 사전 (음식 리뷰)
food_sentiment_dict = korean_sentiment_dict.copy()

# 음식 도메인 추가 단어
food_words = {
    # 긍정
    '신선하다': 2, '신선해요': 2, '신선한': 2,
    '바삭하다': 1, '바삭해요': 1, '바삭한': 1,
    '촉촉하다': 1, '촉촉해요': 1,
    '풍미': 2, '고소하다': 1, '달콤하다': 1,
    '양많다': 1, '가성비': 1, '푸짐하다': 2,
    '분위기좋다': 1,
    
    # 부정
    '짜다': -1, '짜요': -1, '짠': -1,
    '느끼하다': -1, '느끼해요': -1,
    '식었다': -1, '식어요': -1, '차갑다': -1,
    '양적다': -1, '양없다': -1,
    '비싸다': -1, '비싸요': -1,
    '오래기다렸다': -1, '웨이팅': -0.5
}

food_sentiment_dict.update(food_words)

# 음식 리뷰 테스트
food_reviews = [
    "파스타가 정말 맛있어요! 면이 촉촉하고 양도 푸짐해요.",
    "피자가 너무 짜고 느끼해요. 기름이 둥둥.",
    "분위기는 좋은데 가격이 좀 비싸요.",
    "신선한 재료로 만든 샐러드! 최고!"
]

print("음식 리뷰 감성 분석 (도메인 특화 사전)")
print("=" * 60)

for review in food_reviews:
    result = analyze_korean_sentiment(review, food_sentiment_dict)
    print(f"\n리뷰: {review}")
    print(f"  점수: {result['score']:.3f}, 분류: {result['sentiment']}")
    print(f"  키워드: {[w[0] for w in result['keywords']]}")

---

## 1.5 긍정/부정/중립 분류 함수

### 종합 분류 함수 구현

In [ ]:
class SentimentAnalyzer:
    """
    다국어 감성 분석기
    - 영어: VADER 사용
    - 한국어: 커스텀 사전 사용
    """
    
    def __init__(self, korean_dict=None):
        self.korean_dict = korean_dict or korean_sentiment_dict
        if vader_available:
            self.vader = SentimentIntensityAnalyzer()
        else:
            self.vader = None
    
    def detect_language(self, text):
        """간단한 언어 감지"""
        korean_chars = len(re.findall('[가-힣]', text))
        english_chars = len(re.findall('[a-zA-Z]', text))
        return 'korean' if korean_chars > english_chars else 'english'
    
    def analyze(self, text):
        """텍스트 감성 분석"""
        lang = self.detect_language(text)
        
        if lang == 'english' and self.vader:
            scores = self.vader.polarity_scores(text)
            score = scores['compound']
        else:
            result = analyze_korean_sentiment(text, self.korean_dict)
            score = result['score']
        
        # 분류
        if score >= 0.05:
            sentiment = '긍정'
            label = 'positive'
        elif score <= -0.05:
            sentiment = '부정'
            label = 'negative'
        else:
            sentiment = '중립'
            label = 'neutral'
        
        return {
            'text': text,
            'language': lang,
            'score': score,
            'sentiment_kr': sentiment,
            'sentiment_en': label
        }
    
    def analyze_batch(self, texts):
        """여러 텍스트 일괄 분석"""
        return [self.analyze(text) for text in texts]


# 사용 예시
analyzer = SentimentAnalyzer()

mixed_texts = [
    "This product is amazing!",
    "이 제품 정말 좋아요!",
    "Terrible experience.",
    "최악이에요. 실망입니다.",
    "It's okay.",
    "그냥 보통이에요."
]

print("다국어 감성 분석 결과")
print("=" * 70)

results = analyzer.analyze_batch(mixed_texts)
for r in results:
    print(f"\n{r['text']}")
    print(f"  언어: {r['language']}, 점수: {r['score']:.3f}, 감성: {r['sentiment_kr']}")

---

# Part 2: 감성 분석 시각화와 응용

---

## 2.1 pandas로 감성 점수 집계

### 제품별/카테고리별 감성 집계

In [ ]:
# 제품별 감성 집계
product_sentiment = korean_reviews.groupby('product').agg({
    'sentiment_score': ['mean', 'std', 'count'],
    'rating': 'mean'
}).round(3)

product_sentiment.columns = ['평균_감성점수', '감성점수_표준편차', '리뷰수', '평균_평점']
product_sentiment = product_sentiment.sort_values('평균_감성점수', ascending=False)

print("제품별 감성 분석 요약")
print(product_sentiment)

In [ ]:
# 감성별 리뷰 수 집계
sentiment_counts = korean_reviews['sentiment'].value_counts()
sentiment_by_product = korean_reviews.groupby(['product', 'sentiment']).size().unstack(fill_value=0)

print("전체 감성 분포")
print(sentiment_counts)
print("\n제품별 감성 분포")
print(sentiment_by_product)

In [ ]:
# 평점과 감성 점수 상관관계
correlation = korean_reviews['rating'].corr(korean_reviews['sentiment_score'])
print(f"평점과 감성 점수의 상관계수: {correlation:.3f}")

# 평점별 평균 감성 점수
rating_sentiment = korean_reviews.groupby('rating')['sentiment_score'].agg(['mean', 'count'])
print("\n평점별 평균 감성 점수")
print(rating_sentiment)

### 💡 실무 예시: VOC(Voice of Customer) 분석

In [ ]:
# VOC 분석 리포트 생성
def generate_voc_report(df, product_col='product', sentiment_col='sentiment', score_col='sentiment_score'):
    """
    VOC 분석 리포트 생성
    """
    report = {}
    
    # 전체 요약
    total = len(df)
    positive = (df[sentiment_col] == '긍정').sum()
    negative = (df[sentiment_col] == '부정').sum()
    neutral = (df[sentiment_col] == '중립').sum()
    
    report['전체 요약'] = {
        '총 리뷰 수': total,
        '긍정 비율': f"{positive/total*100:.1f}%",
        '부정 비율': f"{negative/total*100:.1f}%",
        '중립 비율': f"{neutral/total*100:.1f}%",
        '평균 감성 점수': f"{df[score_col].mean():.3f}"
    }
    
    # 제품별 요약
    product_summary = df.groupby(product_col).agg({
        score_col: 'mean',
        sentiment_col: lambda x: (x == '긍정').sum() / len(x) * 100
    }).round(1)
    product_summary.columns = ['평균 점수', '긍정률(%)']
    report['제품별 요약'] = product_summary
    
    # 개선 필요 제품 (부정 비율 높은)
    needs_improvement = df[df[sentiment_col] == '부정'].groupby(product_col).size()
    report['개선 필요'] = needs_improvement.sort_values(ascending=False)
    
    return report

# VOC 리포트 생성
voc_report = generate_voc_report(korean_reviews)

print("VOC 분석 리포트")
print("=" * 50)
print("\n[전체 요약]")
for k, v in voc_report['전체 요약'].items():
    print(f"  {k}: {v}")

print("\n[제품별 요약]")
print(voc_report['제품별 요약'])

print("\n[개선 필요 제품 (부정 리뷰 수)]")
print(voc_report['개선 필요'])

---

## 2.2 Plotly로 감성 분포 시각화

### 파이 차트: 전체 감성 분포

In [ ]:
# 감성 분포 파이 차트
sentiment_counts = korean_reviews['sentiment'].value_counts()

# 색상 정의
colors = {'긍정': '#2ecc71', '부정': '#e74c3c', '중립': '#95a5a6'}

fig = px.pie(
    values=sentiment_counts.values,
    names=sentiment_counts.index,
    title='전체 리뷰 감성 분포',
    color=sentiment_counts.index,
    color_discrete_map=colors,
    hole=0.4  # 도넛 차트
)

fig.update_traces(
    textposition='inside',
    textinfo='percent+label+value',
    textfont_size=14
)

fig.update_layout(
    width=600,
    height=450,
    annotations=[dict(text='감성 분포', x=0.5, y=0.5, font_size=16, showarrow=False)]
)

fig.show()

### 막대 차트: 제품별 감성 비교

In [ ]:
# 제품별 감성 점수 막대 차트
product_scores = korean_reviews.groupby('product')['sentiment_score'].mean().sort_values()

# 색상: 점수에 따라 (긍정=초록, 부정=빨강)
bar_colors = ['#2ecc71' if x >= 0 else '#e74c3c' for x in product_scores.values]

fig = go.Figure()

fig.add_trace(go.Bar(
    x=product_scores.index,
    y=product_scores.values,
    marker_color=bar_colors,
    text=[f'{v:.2f}' for v in product_scores.values],
    textposition='outside'
))

# 기준선 추가
fig.add_hline(y=0, line_dash='dash', line_color='gray')

fig.update_layout(
    title='제품별 평균 감성 점수',
    xaxis_title='제품',
    yaxis_title='감성 점수 (-1 ~ 1)',
    yaxis=dict(range=[-1, 1]),
    width=700,
    height=450
)

fig.show()

In [ ]:
# 제품별 감성 분포 누적 막대 차트
sentiment_by_product = korean_reviews.groupby(['product', 'sentiment']).size().unstack(fill_value=0)
# 순서 지정
sentiment_order = ['긍정', '중립', '부정']
sentiment_by_product = sentiment_by_product.reindex(columns=sentiment_order, fill_value=0)

fig = go.Figure()

for sentiment in sentiment_order:
    fig.add_trace(go.Bar(
        x=sentiment_by_product.index,
        y=sentiment_by_product[sentiment],
        name=sentiment,
        marker_color=colors[sentiment]
    ))

fig.update_layout(
    title='제품별 감성 분포 (리뷰 수)',
    xaxis_title='제품',
    yaxis_title='리뷰 수',
    barmode='stack',
    legend_title='감성',
    width=700,
    height=450
)

fig.show()

### 💡 실무 예시: 브랜드 평판 대시보드

In [ ]:
# 종합 감성 분석 대시보드
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'pie'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'scatter'}]],
    subplot_titles=('전체 감성 분포', '제품별 평균 점수', 
                   '제품별 감성 분포', '평점 vs 감성 점수')
)

# 1. 파이 차트
fig.add_trace(
    go.Pie(
        labels=sentiment_counts.index,
        values=sentiment_counts.values,
        marker_colors=[colors[s] for s in sentiment_counts.index],
        hole=0.3
    ),
    row=1, col=1
)

# 2. 제품별 평균 점수
fig.add_trace(
    go.Bar(
        x=product_scores.index,
        y=product_scores.values,
        marker_color=bar_colors,
        showlegend=False
    ),
    row=1, col=2
)

# 3. 제품별 감성 분포 (누적)
for sentiment in sentiment_order:
    fig.add_trace(
        go.Bar(
            x=sentiment_by_product.index,
            y=sentiment_by_product[sentiment],
            name=sentiment,
            marker_color=colors[sentiment],
            showlegend=True
        ),
        row=2, col=1
    )

# 4. 평점 vs 감성 점수 산점도
fig.add_trace(
    go.Scatter(
        x=korean_reviews['rating'],
        y=korean_reviews['sentiment_score'],
        mode='markers',
        marker=dict(
            size=10,
            color=korean_reviews['sentiment_score'],
            colorscale='RdYlGn',
            showscale=True
        ),
        showlegend=False
    ),
    row=2, col=2
)

fig.update_layout(
    title_text='브랜드 평판 분석 대시보드',
    height=800,
    width=1000,
    barmode='stack'
)

# 축 레이블
fig.update_xaxes(title_text='제품', row=1, col=2)
fig.update_yaxes(title_text='감성 점수', row=1, col=2)
fig.update_xaxes(title_text='제품', row=2, col=1)
fig.update_yaxes(title_text='리뷰 수', row=2, col=1)
fig.update_xaxes(title_text='평점', row=2, col=2)
fig.update_yaxes(title_text='감성 점수', row=2, col=2)

fig.show()

---

## 2.3 시계열 감성 트렌드 분석

In [ ]:
# 시계열 데이터 생성 (날짜별 리뷰)
np.random.seed(42)

# 30일간의 리뷰 데이터 시뮬레이션
dates = pd.date_range('2024-01-01', periods=30, freq='D')
n_reviews_per_day = np.random.randint(5, 15, size=30)

time_series_reviews = []
for date, n in zip(dates, n_reviews_per_day):
    for _ in range(n):
        # 감성 점수 시뮬레이션 (시간에 따라 변화)
        base_score = 0.2 + 0.3 * np.sin(date.day / 10)  # 주기적 변화
        score = base_score + np.random.normal(0, 0.3)
        score = max(-1, min(1, score))  # 클리핑
        
        sentiment = '긍정' if score > 0.1 else ('부정' if score < -0.1 else '중립')
        
        time_series_reviews.append({
            'date': date,
            'sentiment_score': score,
            'sentiment': sentiment
        })

ts_df = pd.DataFrame(time_series_reviews)
print(f"총 리뷰 수: {len(ts_df)}")
print(ts_df.head(10))

In [ ]:
# 일별 감성 점수 추이
daily_sentiment = ts_df.groupby('date').agg({
    'sentiment_score': ['mean', 'std', 'count']
}).round(3)
daily_sentiment.columns = ['평균_점수', '표준편차', '리뷰수']
daily_sentiment = daily_sentiment.reset_index()

# 이동 평균 추가
daily_sentiment['MA7'] = daily_sentiment['평균_점수'].rolling(window=7, min_periods=1).mean()

fig = go.Figure()

# 일별 점수
fig.add_trace(go.Scatter(
    x=daily_sentiment['date'],
    y=daily_sentiment['평균_점수'],
    mode='lines+markers',
    name='일별 평균',
    line=dict(color='#3498db', width=1),
    marker=dict(size=6)
))

# 7일 이동평균
fig.add_trace(go.Scatter(
    x=daily_sentiment['date'],
    y=daily_sentiment['MA7'],
    mode='lines',
    name='7일 이동평균',
    line=dict(color='#e74c3c', width=3)
))

# 기준선
fig.add_hline(y=0, line_dash='dash', line_color='gray')

fig.update_layout(
    title='일별 감성 점수 추이 (7일 이동평균)',
    xaxis_title='날짜',
    yaxis_title='감성 점수',
    yaxis=dict(range=[-1, 1]),
    width=900,
    height=450,
    hovermode='x unified'
)

fig.show()

In [ ]:
# 일별 감성 비율 변화 (스택 영역 차트)
daily_sentiment_dist = ts_df.groupby(['date', 'sentiment']).size().unstack(fill_value=0)
daily_sentiment_pct = daily_sentiment_dist.div(daily_sentiment_dist.sum(axis=1), axis=0) * 100

fig = go.Figure()

for sentiment in ['긍정', '중립', '부정']:
    if sentiment in daily_sentiment_pct.columns:
        fig.add_trace(go.Scatter(
            x=daily_sentiment_pct.index,
            y=daily_sentiment_pct[sentiment],
            mode='lines',
            name=sentiment,
            line=dict(color=colors[sentiment]),
            stackgroup='one',
            fillcolor=colors[sentiment]
        ))

fig.update_layout(
    title='일별 감성 비율 변화 (%)',
    xaxis_title='날짜',
    yaxis_title='비율 (%)',
    yaxis=dict(range=[0, 100]),
    width=900,
    height=450,
    hovermode='x unified'
)

fig.show()

---

## 2.4 머신러닝 기반 감성 분석 소개

### 사전 기반 vs ML 기반 비교

| 구분 | 사전 기반 | ML 기반 |
|------|----------|--------|
| 준비 시간 | 빠름 (사전만 있으면) | 느림 (라벨 데이터 필요) |
| 정확도 | 보통 (70-80%) | 높음 (85-95%) |
| 도메인 적응 | 사전 업데이트 필요 | 재학습 필요 |
| 해석 가능성 | 높음 (키워드 확인) | 낮음 (블랙박스) |
| 새로운 표현 | 처리 어려움 | 학습 데이터에 따라 가능 |

In [ ]:
# ML 기반 감성 분석 맛보기 (scikit-learn)
# 실제로는 Week 4에서 자세히 다룹니다

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# 간단한 라벨 데이터 (실제로는 더 많은 데이터 필요)
ml_data = pd.DataFrame({
    'text': [
        '정말 좋아요', '최고입니다', '만족해요', '추천합니다', '훌륭해요',
        '별로예요', '실망이에요', '최악입니다', '후회해요', '나빠요',
        '그냥 그래요', '보통이에요', '괜찮아요', '무난해요', '평범해요'
    ],
    'label': ['긍정']*5 + ['부정']*5 + ['중립']*5
})

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(ml_data['text'])
y = ml_data['label']

# 모델 학습 (데이터가 적어서 전체로 학습)
model = MultinomialNB()
model.fit(X, y)

# 새로운 텍스트 예측
new_texts = ['이 제품 진짜 좋아요', '별로네요 실망', '그냥 평범해요']
new_X = vectorizer.transform(new_texts)
predictions = model.predict(new_X)
probabilities = model.predict_proba(new_X)

print("ML 기반 감성 분석 예측 결과")
print("=" * 50)
for text, pred, prob in zip(new_texts, predictions, probabilities):
    print(f"\n텍스트: {text}")
    print(f"  예측: {pred}")
    print(f"  확률: {dict(zip(model.classes_, prob.round(3)))}")

### 💡 실무 팁: 언제 어떤 방법을 사용할까?

| 상황 | 권장 방법 |
|------|----------|
| 빠른 초기 분석 | 사전 기반 |
| 라벨 데이터 없음 | 사전 기반 |
| 높은 정확도 필요 | ML 기반 |
| 실시간 대량 처리 | 사전 기반 (속도) |
| 특정 도메인 | 커스텀 사전 + ML |

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 감성 점수로 분류하기 ⭐

**문제**: 다음 감성 점수들을 긍정/부정/중립으로 분류하는 함수를 작성하세요.

- 0.05 이상: 긍정
- -0.05 이하: 부정
- 그 외: 중립

```python
scores = [0.8, -0.3, 0.02, -0.7, 0.5, 0.0, -0.04]
```

**기대 결과**: `['긍정', '부정', '중립', '부정', '긍정', '중립', '중립']`

In [ ]:
scores = [0.8, -0.3, 0.02, -0.7, 0.5, 0.0, -0.04]

# 여기에 코드를 작성하세요


### Q2. 감성 사전으로 점수 계산하기 ⭐⭐

**문제**: 주어진 감성 사전을 사용하여 텍스트의 감성 점수를 계산하세요.

```python
simple_dict = {'좋다': 1, '나쁘다': -1, '최고': 2, '최악': -2, '보통': 0}
texts = ['이 음식 최고다', '서비스가 나쁘다', '그냥 보통이다']
```

**출력**: 각 텍스트의 감성 점수

In [ ]:
simple_dict = {'좋다': 1, '나쁘다': -1, '최고': 2, '최악': -2, '보통': 0}
texts = ['이 음식 최고다', '서비스가 나쁘다', '그냥 보통이다']

# 여기에 코드를 작성하세요


### Q3. pandas로 감성 분포 집계하기 ⭐⭐

**문제**: 다음 리뷰 데이터에서 감성별 리뷰 수와 비율을 계산하세요.

```python
reviews_df = pd.DataFrame({
    'review': ['좋아요', '별로예요', '최고!', '그냥 그래요', '실망이에요', '만족해요'],
    'sentiment': ['긍정', '부정', '긍정', '중립', '부정', '긍정']
})
```

**기대 결과**: 감성별 개수와 비율(%) 출력

In [ ]:
reviews_df = pd.DataFrame({
    'review': ['좋아요', '별로예요', '최고!', '그냥 그래요', '실망이에요', '만족해요'],
    'sentiment': ['긍정', '부정', '긍정', '중립', '부정', '긍정']
})

# 여기에 코드를 작성하세요


### Q4. Plotly 파이 차트 그리기 ⭐⭐⭐

**문제**: Q3의 감성 분포를 Plotly 파이 차트로 시각화하세요.

**요구사항**:
- 긍정=초록, 부정=빨강, 중립=회색
- 퍼센트와 레이블 표시
- 도넛 차트 형태 (hole=0.3)

In [ ]:
# Q3의 데이터 사용

# 여기에 코드를 작성하세요


### Q5. 카테고리별 감성 비교 막대 차트 ⭐⭐⭐

**문제**: 다음 데이터를 사용하여 카테고리별 평균 감성 점수 막대 차트를 그리세요.

```python
category_data = pd.DataFrame({
    'category': ['음식', '음식', '서비스', '서비스', '가격', '가격'],
    'sentiment_score': [0.8, 0.6, -0.3, 0.2, 0.4, -0.5]
})
```

**요구사항**:
- 평균 점수 계산
- 양수=초록, 음수=빨강 색상
- 0 기준선 표시

In [ ]:
category_data = pd.DataFrame({
    'category': ['음식', '음식', '서비스', '서비스', '가격', '가격'],
    'sentiment_score': [0.8, 0.6, -0.3, 0.2, 0.4, -0.5]
})

# 여기에 코드를 작성하세요


---

## Part 2 심화 퀴즈

---

### Q6. 강조어 처리하는 감성 분석 함수 ⭐⭐⭐

**문제**: 강조어(매우, 정말, 너무)가 있으면 점수를 1.5배 하는 감성 분석 함수를 작성하세요.

```python
sentiment_dict = {'좋다': 1, '나쁘다': -1}
texts = ['좋다', '매우 좋다', '정말 나쁘다', '나쁘다']
```

**기대 결과**: `[1, 1.5, -1.5, -1]`

In [ ]:
sentiment_dict = {'좋다': 1, '나쁘다': -1}
texts = ['좋다', '매우 좋다', '정말 나쁘다', '나쁘다']

# 여기에 코드를 작성하세요


### Q7. 제품별 긍정률 계산 및 순위 ⭐⭐⭐

**문제**: 제품별 긍정률(긍정 리뷰 / 전체 리뷰 * 100)을 계산하고 높은 순서로 정렬하세요.

```python
product_reviews = pd.DataFrame({
    'product': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'C'],
    'sentiment': ['긍정', '긍정', '부정', '긍정', '부정', '부정', '긍정', '긍정', '긍정', '중립']
})
```

**기대 결과**: 제품별 긍정률(%) 내림차순 정렬

In [ ]:
product_reviews = pd.DataFrame({
    'product': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'C'],
    'sentiment': ['긍정', '긍정', '부정', '긍정', '부정', '부정', '긍정', '긍정', '긍정', '중립']
})

# 여기에 코드를 작성하세요


### Q8. 누적 막대 차트로 감성 분포 시각화 ⭐⭐⭐⭐

**문제**: Q7의 데이터를 사용하여 제품별 감성 분포 누적 막대 차트를 그리세요.

**요구사항**:
- 긍정=초록, 부정=빨강, 중립=회색
- 비율(%) 기준으로 표시
- barmode='stack'

In [ ]:
# Q7의 데이터 사용

# 여기에 코드를 작성하세요


### Q9. 시계열 감성 트렌드 분석 ⭐⭐⭐⭐

**문제**: 다음 일별 데이터를 사용하여 감성 점수 추이 라인 차트를 그리세요. 3일 이동평균도 함께 표시하세요.

```python
daily_data = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=10),
    'sentiment_score': [0.3, 0.5, 0.2, -0.1, -0.3, 0.1, 0.4, 0.6, 0.3, 0.5]
})
```

**요구사항**:
- 일별 점수: 파란색 선 + 마커
- 3일 이동평균: 빨간색 굵은 선
- y=0 기준선 표시

In [ ]:
daily_data = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=10),
    'sentiment_score': [0.3, 0.5, 0.2, -0.1, -0.3, 0.1, 0.4, 0.6, 0.3, 0.5]
})

# 여기에 코드를 작성하세요


### Q10. 종합 VOC 분석 대시보드 ⭐⭐⭐⭐⭐

**문제**: 다음 VOC 데이터를 분석하고 종합 대시보드를 만드세요.

```python
voc_data = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=20).repeat(3),
    'category': ['제품', '서비스', '배송'] * 20,
    'review': ['좋아요 만족합니다', '별로예요', '빨라요'] * 20,
    'sentiment_score': np.random.uniform(-1, 1, 60)
})
```

**분석 항목**:
1. 전체 감성 분포 (파이 차트)
2. 카테고리별 평균 점수 (막대 차트)
3. 일별 감성 추이 (라인 차트)
4. 카테고리별 부정 비율 (막대 차트)

**요구사항**:
- `make_subplots`로 2x2 레이아웃
- 적절한 색상과 레이블
- 인사이트 3개 이상 도출

In [ ]:
np.random.seed(42)
voc_data = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=20).repeat(3),
    'category': ['제품', '서비스', '배송'] * 20,
    'review': ['좋아요 만족합니다', '별로예요', '빨라요'] * 20,
    'sentiment_score': np.random.uniform(-1, 1, 60)
})

# 감성 분류 추가
voc_data['sentiment'] = voc_data['sentiment_score'].apply(
    lambda x: '긍정' if x > 0.1 else ('부정' if x < -0.1 else '중립')
)

# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 사전 기반 감성 분석 핵심 요약

| 개념 | 도구/방법 | 활용 |
|------|----------|------|
| VADER | `SentimentIntensityAnalyzer` | 영어 SNS 텍스트 분석 |
| 한국어 사전 | KNU 감성 사전, 커스텀 사전 | 한글 리뷰 분석 |
| 감성 분류 | compound 점수 기준 | 긍정/부정/중립 |
| 강조어 처리 | 점수 배수 적용 | "매우 좋다" 처리 |
| 부정어 처리 | 점수 반전 | "안 좋다" 처리 |

### Part 2: 감성 분석 시각화 핵심 요약

| 시각화 | Plotly 함수 | 언제 쓰나? |
|--------|------------|----------|
| 파이 차트 | `px.pie()` | 감성 분포 비율 |
| 막대 차트 | `go.Bar()` | 카테고리별 비교 |
| 누적 막대 | `barmode='stack'` | 구성 비율 비교 |
| 라인 차트 | `go.Scatter()` | 시계열 트렌드 |
| 대시보드 | `make_subplots()` | 종합 분석 |

### 💡 실무 팁

1. **도메인 특화 사전**: 일반 사전 + 도메인 단어로 정확도 향상
2. **강조어/부정어**: 반드시 처리해야 정확한 분석 가능
3. **평점 vs 감성**: 상관관계 확인하여 분석 신뢰도 검증
4. **시계열 분석**: 이동평균으로 노이즈 제거, 트렌드 파악
5. **VOC 대시보드**: 한눈에 현황 파악, 개선점 도출
6. **ML 병행**: 사전 기반으로 초기 분석, ML로 정확도 개선
7. **실시간 모니터링**: 부정 급증 시 알림 설정 권장